# Day 9

## Part 1

We need to edit our Intcode interpreter to support a new parameter mode called "relative".

> Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.
>
> The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.
>
> The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

Moreover, the interperter should be able to write in memory *after* the program, and to support large number (which comes for free in python).

In [1]:
%autoawait trio

In [2]:
from collections import defaultdict

import trio


class Computer:
    def __init__(self, program):
        self.position = 0
        self.relative_base = 0
        self.parameters_mode = None
        self.program = program.copy()

    def set_input_output(self, buffer_max=10):
        input_write, input_read = trio.open_memory_channel(buffer_max)
        output_write, output_read = trio.open_memory_channel(buffer_max)
        
        self.input, self.output = input_read, output_write
        
        return input_write, output_read
        
    @staticmethod
    def log(*args):
        if False:
            print(*args)

    def get_param_value(self, param):
        param_address = self.get_param_address(param)
        
        try:
            value = self.program[param_address]
        except IndexError:
            self.extend(param_address)
            value = self.program[param_address]
            
        return value

    def get_param_address(self, param):
        mode = self.parameters_mode[param]

        if mode == 0:  # position mode
            return self.program[self.position + param]
        elif mode == 1:  # immediate mode
            return self.position + param
        elif mode == 2:  # relative mode:
            param_position = self.position + param
            return self.relative_base + self.program[param_position]
        else:
            raise ValueError("Unknown paremeter mode")

    def read_instruction(self):
        # read instruction and split it into opcode and parameters mode
        try:
            instruction = self.program[self.position]
        except IndexError:
            # end of program
            return None

        opcode = instruction % 100
        self.parameters_mode = defaultdict(lambda: 0)

        value = instruction // 100
        i = 1
        while value > 0:
            self.parameters_mode[i] = value % 10
            value //= 10
            i += 1

        self.log(self.position, '>', instruction, opcode, dict(self.parameters_mode))
        return opcode
    
    def write(self, address, value):
        try:
            self.program[address] = value
        except IndexError:
            self.extend(address)
            self.program[address] = value
            
    def extend(self, address):
        self.program.extend([0] * (1 + address - len(self.program)))

    async def compute_close(self):
        """Run the programm, but close the output when terminated."""
        async with self.output:
            await self.compute()
            
        
    async def compute(self):
        """Compute the final state of a program, and return it.

        `program` is a list of int.
        `read_input` is a method returning a value each time it is called.
        `write_output` is a method taking a value as parameter.
        """
        # init state
        self.position = 0

        # run program
        while True:
            opcode = self.read_instruction()
            if opcode is None:
                # end of program
                log("Stop: EOF")
                return

            # execute opcode
            if opcode == 1:
                # add x y
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                result = x + y
                self.write(result_addr, result)
                self.log("add", x, y, result_addr, result)

                step = 4
            elif opcode == 2:
                # mult x y
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                result = x * y
                self.write(result_addr, result)
                self.log("mult", x, y, result_addr, result)

                step = 4
            elif opcode == 3:
                # read x
                result_addr = self.get_param_address(1)

                value = await self.input.receive()
                self.write(result_addr, value)
                self.log("read", value, result_addr)

                step = 2
            elif opcode == 4:
                # write x
                value = self.get_param_value(1)
                await self.output.send(value)
                self.log("write", value)

                step = 2
            elif opcode == 5:
                # jump-if-true x y
                value = self.get_param_value(1)
                jump_addr = self.get_param_value(2)

                self.log("jump-if-true", value, jump_addr)

                if value:
                    self.position = jump_addr
                    step = 0
                else:
                    step = 3
            elif opcode == 6:
                # jump-if-false x y
                value = self.get_param_value(1)
                jump_addr = self.get_param_value(2)

                self.log("jump-if-false", value, jump_addr)

                if not value:
                    self.position = jump_addr
                    step = 0
                else:
                    step = 3
            elif opcode == 7:
                # lt x y z
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                self.log("lt", x, y, result_addr)

                if x < y:
                    self.write(result_addr, 1)
                else:
                    self.write(result_addr, 0)

                step = 4
            elif opcode == 8:
                # eq x y z
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                self.log("eq", x, y, result_addr)

                if x == y:
                    self.write(result_addr, 1)
                else:
                    self.write(result_addr, 0)

                step = 4
            elif opcode == 9:
                # relative_base_add x
                x = self.get_param_value(1)
                self.relative_base += x
                
                self.log("relative_base_add", x)
                
                step = 2
            elif opcode == 99:
                # end of program
                self.log("stop")
                break
            else:
                # unknown instruction
                self.log("unknown opcode")
                break

            self.position += step

        return self.program

### Tests

Read input and return the same value.

In [3]:
program = [3,0,4,0,99]
computer = Computer(program)
input_, output = computer.set_input_output()
await input_.send(42)
await computer.compute()
await output.receive()

42

Take no input and return itself as output.

In [4]:
program = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
computer = Computer(program)
input_, output = computer.set_input_output()

async def print_result():
    async with output:
        async for value in output:
            print(value, end=", ")

async with trio.open_nursery() as nursery:
    nursery.start_soon(computer.compute_close)
    nursery.start_soon(print_result)

109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99, 

Output a 16-digit number.

In [5]:
program = [1102,34915192,34915192,7,4,7,99,0]
computer = Computer(program)
input_, output = computer.set_input_output()

async def print_result():
    async with output:
        async for value in output:
            print(value, len(str(value)))

async with trio.open_nursery() as nursery:
    nursery.start_soon(computer.compute_close)
    nursery.start_soon(print_result)

1219070632396864 16


Should output the large number in the middle.

In [6]:
program = [104,1125899906842624,99]
computer = Computer(program)
input_, output = computer.set_input_output()

async def print_result():
    async with output:
        async for value in output:
            print(value)

async with trio.open_nursery() as nursery:
    nursery.start_soon(computer.compute_close)
    nursery.start_soon(print_result)

1125899906842624


### Result

This programm will run some checks, and output any malfunctioning opcode.
As our interpreter is fully functionnal, it should output no opcode but a single keycode.

In [7]:
program = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,3,1,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1101,0,31,1019,1101,25,0,1008,1102,35,1,1009,1102,422,1,1029,1102,1,21,1005,1102,1,734,1027,1102,29,1,1000,1101,32,0,1018,1102,28,1,1016,1101,0,38,1015,1101,0,378,1023,1101,30,0,1017,1102,1,381,1022,1101,0,37,1006,1102,1,1,1021,1101,0,24,1011,1102,1,23,1002,1101,0,0,1020,1101,0,20,1007,1101,427,0,1028,1101,26,0,1014,1101,27,0,1010,1101,0,39,1001,1101,34,0,1012,1102,1,36,1013,1101,0,33,1003,1101,804,0,1025,1101,737,0,1026,1102,1,809,1024,1102,1,22,1004,109,9,1201,-7,0,63,1008,63,20,63,1005,63,205,1001,64,1,64,1106,0,207,4,187,1002,64,2,64,109,2,21102,40,1,1,1008,1012,40,63,1005,63,233,4,213,1001,64,1,64,1106,0,233,1002,64,2,64,109,4,1208,-7,25,63,1005,63,255,4,239,1001,64,1,64,1106,0,255,1002,64,2,64,109,-24,1207,10,38,63,1005,63,271,1105,1,277,4,261,1001,64,1,64,1002,64,2,64,109,25,21107,41,40,-3,1005,1013,293,1105,1,299,4,283,1001,64,1,64,1002,64,2,64,109,5,1205,-1,311,1106,0,317,4,305,1001,64,1,64,1002,64,2,64,109,-23,1202,6,1,63,1008,63,22,63,1005,63,339,4,323,1105,1,343,1001,64,1,64,1002,64,2,64,109,1,2101,0,2,63,1008,63,37,63,1005,63,367,1001,64,1,64,1106,0,369,4,349,1002,64,2,64,109,29,2105,1,-5,1106,0,387,4,375,1001,64,1,64,1002,64,2,64,109,-26,2101,0,0,63,1008,63,23,63,1005,63,409,4,393,1106,0,413,1001,64,1,64,1002,64,2,64,109,26,2106,0,0,4,419,1106,0,431,1001,64,1,64,1002,64,2,64,109,-17,21108,42,42,6,1005,1017,453,4,437,1001,64,1,64,1106,0,453,1002,64,2,64,109,7,21101,43,0,-8,1008,1010,44,63,1005,63,477,1001,64,1,64,1105,1,479,4,459,1002,64,2,64,109,-7,1206,10,495,1001,64,1,64,1106,0,497,4,485,1002,64,2,64,109,-5,2108,36,0,63,1005,63,513,1106,0,519,4,503,1001,64,1,64,1002,64,2,64,109,3,2102,1,-5,63,1008,63,22,63,1005,63,541,4,525,1105,1,545,1001,64,1,64,1002,64,2,64,109,3,1207,-6,38,63,1005,63,567,4,551,1001,64,1,64,1105,1,567,1002,64,2,64,109,-15,2107,20,8,63,1005,63,585,4,573,1106,0,589,1001,64,1,64,1002,64,2,64,109,-1,1208,5,36,63,1005,63,609,1001,64,1,64,1106,0,611,4,595,1002,64,2,64,109,30,21101,44,0,-7,1008,1019,44,63,1005,63,633,4,617,1106,0,637,1001,64,1,64,1002,64,2,64,109,-25,1201,0,0,63,1008,63,39,63,1005,63,659,4,643,1105,1,663,1001,64,1,64,1002,64,2,64,109,27,1206,-8,677,4,669,1106,0,681,1001,64,1,64,1002,64,2,64,109,-28,2108,29,0,63,1005,63,703,4,687,1001,64,1,64,1106,0,703,1002,64,2,64,109,5,21107,45,46,7,1005,1012,725,4,709,1001,64,1,64,1106,0,725,1002,64,2,64,109,30,2106,0,-8,1105,1,743,4,731,1001,64,1,64,1002,64,2,64,109,-22,21102,46,1,4,1008,1017,44,63,1005,63,767,1001,64,1,64,1105,1,769,4,749,1002,64,2,64,109,-15,1202,10,1,63,1008,63,23,63,1005,63,793,1001,64,1,64,1106,0,795,4,775,1002,64,2,64,109,19,2105,1,7,4,801,1105,1,813,1001,64,1,64,1002,64,2,64,109,6,1205,-2,827,4,819,1106,0,831,1001,64,1,64,1002,64,2,64,109,-20,2107,22,2,63,1005,63,851,1001,64,1,64,1106,0,853,4,837,1002,64,2,64,109,20,21108,47,44,-8,1005,1015,869,1105,1,875,4,859,1001,64,1,64,1002,64,2,64,109,-22,2102,1,4,63,1008,63,23,63,1005,63,899,1001,64,1,64,1106,0,901,4,881,4,64,99,21101,0,27,1,21102,915,1,0,1106,0,922,21201,1,28703,1,204,1,99,109,3,1207,-2,3,63,1005,63,964,21201,-2,-1,1,21101,0,942,0,1106,0,922,22101,0,1,-1,21201,-2,-3,1,21101,957,0,0,1105,1,922,22201,1,-1,-2,1105,1,968,21201,-2,0,-2,109,-3,2105,1,0]
computer = Computer(program)
input_, output = computer.set_input_output()

async def print_result():
    async with output:
        async for value in output:
            print(value)

async with trio.open_nursery() as nursery:
    nursery.start_soon(computer.compute_close)
    await input_.send(1)
    nursery.start_soon(print_result)

2204990589


## Part 2

We simply need to run again this program in boost mode by providing it 2 as input.

In [8]:
program = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,3,1,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1101,0,31,1019,1101,25,0,1008,1102,35,1,1009,1102,422,1,1029,1102,1,21,1005,1102,1,734,1027,1102,29,1,1000,1101,32,0,1018,1102,28,1,1016,1101,0,38,1015,1101,0,378,1023,1101,30,0,1017,1102,1,381,1022,1101,0,37,1006,1102,1,1,1021,1101,0,24,1011,1102,1,23,1002,1101,0,0,1020,1101,0,20,1007,1101,427,0,1028,1101,26,0,1014,1101,27,0,1010,1101,0,39,1001,1101,34,0,1012,1102,1,36,1013,1101,0,33,1003,1101,804,0,1025,1101,737,0,1026,1102,1,809,1024,1102,1,22,1004,109,9,1201,-7,0,63,1008,63,20,63,1005,63,205,1001,64,1,64,1106,0,207,4,187,1002,64,2,64,109,2,21102,40,1,1,1008,1012,40,63,1005,63,233,4,213,1001,64,1,64,1106,0,233,1002,64,2,64,109,4,1208,-7,25,63,1005,63,255,4,239,1001,64,1,64,1106,0,255,1002,64,2,64,109,-24,1207,10,38,63,1005,63,271,1105,1,277,4,261,1001,64,1,64,1002,64,2,64,109,25,21107,41,40,-3,1005,1013,293,1105,1,299,4,283,1001,64,1,64,1002,64,2,64,109,5,1205,-1,311,1106,0,317,4,305,1001,64,1,64,1002,64,2,64,109,-23,1202,6,1,63,1008,63,22,63,1005,63,339,4,323,1105,1,343,1001,64,1,64,1002,64,2,64,109,1,2101,0,2,63,1008,63,37,63,1005,63,367,1001,64,1,64,1106,0,369,4,349,1002,64,2,64,109,29,2105,1,-5,1106,0,387,4,375,1001,64,1,64,1002,64,2,64,109,-26,2101,0,0,63,1008,63,23,63,1005,63,409,4,393,1106,0,413,1001,64,1,64,1002,64,2,64,109,26,2106,0,0,4,419,1106,0,431,1001,64,1,64,1002,64,2,64,109,-17,21108,42,42,6,1005,1017,453,4,437,1001,64,1,64,1106,0,453,1002,64,2,64,109,7,21101,43,0,-8,1008,1010,44,63,1005,63,477,1001,64,1,64,1105,1,479,4,459,1002,64,2,64,109,-7,1206,10,495,1001,64,1,64,1106,0,497,4,485,1002,64,2,64,109,-5,2108,36,0,63,1005,63,513,1106,0,519,4,503,1001,64,1,64,1002,64,2,64,109,3,2102,1,-5,63,1008,63,22,63,1005,63,541,4,525,1105,1,545,1001,64,1,64,1002,64,2,64,109,3,1207,-6,38,63,1005,63,567,4,551,1001,64,1,64,1105,1,567,1002,64,2,64,109,-15,2107,20,8,63,1005,63,585,4,573,1106,0,589,1001,64,1,64,1002,64,2,64,109,-1,1208,5,36,63,1005,63,609,1001,64,1,64,1106,0,611,4,595,1002,64,2,64,109,30,21101,44,0,-7,1008,1019,44,63,1005,63,633,4,617,1106,0,637,1001,64,1,64,1002,64,2,64,109,-25,1201,0,0,63,1008,63,39,63,1005,63,659,4,643,1105,1,663,1001,64,1,64,1002,64,2,64,109,27,1206,-8,677,4,669,1106,0,681,1001,64,1,64,1002,64,2,64,109,-28,2108,29,0,63,1005,63,703,4,687,1001,64,1,64,1106,0,703,1002,64,2,64,109,5,21107,45,46,7,1005,1012,725,4,709,1001,64,1,64,1106,0,725,1002,64,2,64,109,30,2106,0,-8,1105,1,743,4,731,1001,64,1,64,1002,64,2,64,109,-22,21102,46,1,4,1008,1017,44,63,1005,63,767,1001,64,1,64,1105,1,769,4,749,1002,64,2,64,109,-15,1202,10,1,63,1008,63,23,63,1005,63,793,1001,64,1,64,1106,0,795,4,775,1002,64,2,64,109,19,2105,1,7,4,801,1105,1,813,1001,64,1,64,1002,64,2,64,109,6,1205,-2,827,4,819,1106,0,831,1001,64,1,64,1002,64,2,64,109,-20,2107,22,2,63,1005,63,851,1001,64,1,64,1106,0,853,4,837,1002,64,2,64,109,20,21108,47,44,-8,1005,1015,869,1105,1,875,4,859,1001,64,1,64,1002,64,2,64,109,-22,2102,1,4,63,1008,63,23,63,1005,63,899,1001,64,1,64,1106,0,901,4,881,4,64,99,21101,0,27,1,21102,915,1,0,1106,0,922,21201,1,28703,1,204,1,99,109,3,1207,-2,3,63,1005,63,964,21201,-2,-1,1,21101,0,942,0,1106,0,922,22101,0,1,-1,21201,-2,-3,1,21101,957,0,0,1105,1,922,22201,1,-1,-2,1105,1,968,21201,-2,0,-2,109,-3,2105,1,0]
computer = Computer(program)
input_, output = computer.set_input_output()

async def print_result():
    async with output:
        async for value in output:
            print(value)

async with trio.open_nursery() as nursery:
    nursery.start_soon(computer.compute_close)
    await input_.send(2)
    nursery.start_soon(print_result)

50008
